In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-02-21 14:32:14--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2024-02-21 14:32:14 (238 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [ ]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [ ]:
df_merged = df_ratings.merge(df_books, on="isbn", how='left')

In [ ]:
df_merged

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5.0,Rites of Passage,Judith Rae
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks
3,276729,052165615X,3.0,Help!: Level 1,Philip Prowse
4,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather
...,...,...,...,...,...
1149775,276704,1563526298,9.0,Get Clark Smart : The Ultimate Guide for the S...,Clark Howard
1149776,276706,0679447156,0.0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil
1149777,276709,0515107662,10.0,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter
1149778,276721,0590442449,10.0,Fourth Grade Rats,Jerry Spinelli


In [ ]:
# Get users who left more than 200 ratings
# Get books that have more than 100 reviews
# For statistic significance

users = df_merged.user.value_counts()
isbn = df_merged.isbn.value_counts()

users = users[users >= 200].index
isbn = isbn[isbn >= 100].index

df = df_merged.loc[(df_merged["user"].isin(users.values)) & (df_merged['isbn'].isin(isbn.values))]

In [ ]:
print(df.shape)
df[df.isnull().any(axis=1)]

(49781, 5)


,user,isbn,rating,title,author
10305,254,0679781587,8.0,NaN,NaN
20906,4017,0679781587,9.0,NaN,NaN
25618,6242,0749397543,3.0,NaN,NaN
28713,6575,0679781587,9.0,NaN,NaN
31250,7286,0679781587,0.0,NaN,NaN
...,...,...,...,...,...
1137944,273979,0679781587,0.0,NaN,NaN
1138125,274004,0091867770,0.0,NaN,NaN
1139622,274061,0679781587,0.0,NaN,NaN
1146374,275970,0091867770,0.0,NaN,NaN


In [ ]:
df = df.drop_duplicates(["title","user"])
df.isna().sum()

user        0
isbn        0
rating      0
title     238
author    238
dtype: int64

In [ ]:
df = df.dropna(how='any')
df.shape

(49136, 5)

In [ ]:
# Create pivot table
df_pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)
df_pivot.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Create a matrix
df_matrix = csr_matrix(df_pivot.values)

# Classifier
nn = NearestNeighbors(metric='cosine')

# Train the classifier
nn.fit(df_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
# Because the query set matches the training set, the nearest neighbor of each point is the point itself, at a distance of zero.
distance, indices = nn.kneighbors([df_pivot.loc["2nd Chance"]], 6, return_distance=True)

#indices[0] -> a list

array([  2, 194, 545, 643,  99, 552])

In [ ]:
books = df_pivot.iloc[indices[0][1:]].index.to_list()
dist = list(distance[0][1:])
for r in zip(books, dist):
  print(r)

('Four Blind Mice', 0.6200344)
('The Next Accident', 0.7210984)
('Violets Are Blue', 0.7334745)
('Cat &amp; Mouse (Alex Cross Novels)', 0.73426455)
('The Perfect Husband', 0.7371001)


In [ ]:
# function to return recommended books - this will be tested
# show 5 nearest neighbors and the distance
def get_recommends(book = ""):

  distance, indices = nn.kneighbors([df_pivot.loc[book]], 6, return_distance=True)
  recom_books = df_pivot.iloc[np.flip(indices[0])[:-1]].index.to_list()   # np.flip is to reverse the order of distance (descending), and the target book will be at the last one
  recom_distance = list(np.flip(distance[0])[:-1])

  recommended_books = [book, []]
  for r in zip(recom_books, recom_distance):
    recommended_books[1].append(r)

  return recommended_books


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [("I'll Be Seeing You", 0.8016211), ('The Weight of Water', 0.77085835), ('The Surgeon', 0.7699411), ('I Know This Much Is True', 0.7677075), ('The Lovely Bones: A Novel', 0.7234864)]]
You passed the challenge! 🎉🎉🎉🎉🎉
